# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [1]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

/Users/jamesrushing/opt/miniconda3/envs/morph/lib/python3.7/site-packages/pyscf/lib/misc.py:50: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
gsd_file = "/Users/jamesrushing/cmelab/data/workspace/35e3dc4320d6ed3424bdba79fae77bce/trajectory.gsd"

with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[0]
    end_snap = f[-1]

    

ref_distance = 3.563594872561358

start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance

box = start_snap.configuration.box[:3]
unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
unwrap_snap = deepcopy(start_snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1)
comp.visualize().show()

#box = end_snap.configuration.box[:3]
#unwrapped_positions = end_snap.particles.position + end_snap.particles.image * box
#unwrap_snap = deepcopy(end_snap)
#unwrap_snap.particles.position = unwrapped_positions
#unwrap_snap.particles.types = [amber_dict[i].symbol for i in end_snap.particles.types]
#comp = from_snapshot(unwrap_snap, scale=0.1)
#comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
print(start_snap.configuration.box)
print(end_snap.configuration.box)

[309.47433 309.47433 309.47433   0.        0.        0.     ]
[62.14245 62.14245 62.14245  0.       0.       0.     ]


below is some scratch for breaking chromophores into pieces of smiles strings. may be useful down the road but for now its easier to explicity prescibe atom indeces for your chromo

In [4]:
#smarts_str = "c1ccccc1"
#aaids=[]
#aaids_pent = chromophores.get_chromo_ids_smiles(snap, "c1sc2ccsc2c1", amber_dict)
#comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)

#aaids_benzene = chromophores.get_chromo_ids_smiles(snap, smarts_str, amber_dict)
#aaids_thiothene = chromophores.get_chromo_ids_smiles(snap, "c1sc(C)cc1", amber_dict)
#aaids.extend(aaids_benzene)
#aaids.extend(aaids_thiothene)
#aaids.extend(aaids_pent)


the indexes below came from viewing the single molecule trajectory in vmd and clicking on the molecules that I like for my chromos. this cell visualizes those chosen. picking the fused ring core as one chromo and the end group as one

In [53]:
chromo_ids_core = np.array([19,162,163,161,101,102,100,99,98,92,91,93,31,30,94,32,30,27,29,28,27,23,24,22,25,20,97])
chromo_ids_ends = np.array([0,7,18,1,2,4,6,15,17,18,14,16,10,13,12,11])
#chromo_ids_ends2 = np.array([177,175,176,174,173,163,165,172,168,166,169,171,170,181,178,180,179,])
for i,p in enumerate(comp.particles()):
    if i in chromo_ids_core:
        p.name = "Kr"
    if i in chromo_ids_ends:
        p.name = "S"
   # if i in chromo_ids_ends2:
   #     p.name = "Kr"
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Trying to get homo and lumo for a single molecule of itic below. using functions in execute_qcc.py. here we use all molecule ids from start_snap.particles.N to get homo lumos for sinlge molecule gsd

displayed : 

HOMO-1, HOMO, LUMO, LUMO+1 above. 

In [55]:
qcc_input = eqcc.write_qcc_inp(start_snap, chromo_ids_core, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


RuntimeError: Electron number 213 and spin 0 are not consistent
Note mol.spin = 2S = Nalpha - Nbeta, not 2S+1

below i want to get the homolumos for just that back bone as taken from a single molecule gsd

In [54]:
qcc_input = eqcc.write_qcc_inp(start_snap, chromo_ids_ends, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


HOMO-1, HOMO, LUMO, LUMO+1
[-6.7930277  -4.78264222  0.49660801  1.1996445 ]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

above this is just the single molecule homo lumo calcs. below i want to bring the build created gsd and get pairwise calcs. the mbuild stuff is being weird . going to just use a two molecule gsd from plankton to begin

In [8]:
chromo2_ids = chromo_ids +186

NameError: name 'chromo_ids' is not defined

In [14]:
aaids=[]
aaids2 = []

In [15]:
aaids.append(chromo_ids_core)
aaids2.append(chromo_ids_ends)
#aaids.append(chromo_ids_core2)

In [16]:
print(aaids)

[array([162, 163, 161, 101, 102, 100,  99,  98,  92,  91,  93,  31,  30,
        94,  32,  30,  27,  29,  28,  27,  23,  24,  22,  25,  20,  97])]


In [37]:
chromo_list = []

chromo_list.append(chromophores.Chromophore(0, start_snap, chromo_ids_ends, "acceptor", amber_dict))
chromo_list.append(chromophores.Chromophore(1, start_snap, chromo_ids_core, "donor", amber_dict))

In [38]:
print(chromo_list)

[Chromophore 0 (acceptor): 16 atoms at -149.215 -137.338 145.534, Chromophore 1 (donor): 26 atoms at -138.774 -140.454 148.184]


In [39]:
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, start_snap, amber_dict, d_cut=100)
print(f"There are {len(qcc_pairs)} chromophore pairs")

There are 1 chromophore pairs


In [40]:
i =  0# try any number from 0 to 180
print(f"Pair #{i}:")
visualize_qcc_input(qcc_pairs[i][1])

Pair #0:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [41]:
%%time
dimer_data = eqcc.dimer_homolumo(qcc_pairs, "two-molecule-test.txt")

CPU times: user 10.5 ms, sys: 33 ms, total: 43.5 ms
Wall time: 3.24 s


In [42]:
%%time
data = eqcc.singles_homolumo(chromo_list, "singles.txt")

CPU times: user 10.9 ms, sys: 33.1 ms, total: 44 ms
Wall time: 2.28 s


In [43]:
%%time
eqcc.set_energyvalues(chromo_list, "singles.txt", "two-molecule-test.txt")

AssertionError: 

In [44]:
print(chromo_list)

[Chromophore 0 (acceptor): 16 atoms at -149.215 -137.338 145.534, Chromophore 1 (donor): 26 atoms at -138.774 -140.454 148.184]


In [45]:
i = 1
chromo = chromo_list[i]
print(f"Chromophore {i}:")
print(f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} LUMO+1: {chromo.lumo_1:.2f}")
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

Chromophore 1:
HOMO-1: -6.66 HOMO: -4.93 LUMO: -1.41 LUMO+1: -0.28
1 neighbors
DeltaE of first neighbor: 5.430


TypeError: unsupported format string passed to NoneType.__format__